In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [2]:
using Distributed

In [3]:
addprocs(2)

2-element Vector{Int64}:
 2
 3

In [4]:
@everywhere using Base: @kwdef
@everywhere using Distributed, DistributedStreams, DistributedArrays

In [5]:
control_messages, control_responses, control = launch_sentinel(;
    workers=[2], verbose=true, buffer_size=1024, timeout=1
)

(RemoteChannel{Channel{ControlMessage}}(1, 1, 19), RemoteChannel{Channel{ControlMessage}}(1, 1, 20), DistributedStreams.RemoteWorkerControl(@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}[(worker = 2, flag = Base.RefValue{Bool}(false))], Any[Future(2, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (8, 130653765041152, 130653764987024)), nothing)]))

In [6]:
sentinel_status_t = @async fetch(only(control.status))
sentinel_running() = ! istaskdone(sentinel_status_t)

sentinel_running (generic function with 1 method)

In [7]:
sentinel_running()

true

In [8]:
ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

@everywhere function test_fn(i)
    println("hi there, I'm running on pid=$(myid())")
    if i == 1
        error("bad input")
    end
    return i+1
end

In [9]:
m_f = ControlMessage(
    message_type=DistributedStreams.start,
    target=3,
    payload=FunctionPayload(
        f=test_fn,
        in=ch_in,
        out=ch_out
    )
)

ControlMessage(DistributedStreams.start, 3, FunctionPayload(test_fn, RemoteChannel{Channel{Int64}}(1, 1, 25), RemoteChannel{Channel{Int64}}(1, 1, 26)))

In [10]:
put!(control_messages, m_f)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 3


In [11]:
collect!(control_responses)

      From worker 2:	Started
      From worker 2:	Worker started for: 3


1-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 3, nothing)

In [12]:
put!(ch_in, 10)

RemoteChannel{Channel{Int64}}(1, 1, 25)

      From worker 3:	hi there, I'm running on pid=3


In [13]:
collect!(ch_out)

1-element Vector{Int64}:
 11

In [14]:
put!(ch_in, 11)

RemoteChannel{Channel{Int64}}(1, 1, 25)

      From worker 3:	hi there, I'm running on pid=3


In [15]:
collect!(ch_out)

1-element Vector{Int64}:
 12

In [16]:
rmprocs(3)

Task (done) @0x000076d4304104c0

      From worker 2:	Worker 3 died
      From worker 2:	Worker 3 completed


In [17]:
collect!(control_responses)

2-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.failed, 3, nothing)
 ControlMessage(DistributedStreams.completed, 3, ReturnPayload(ProcessExitedException(3)))

In [18]:
workers()

1-element Vector{Int64}:
 2

In [20]:
sentinel_running()

true

In [21]:
addprocs(1)
@everywhere using Distributed, DistributedStreams, DistributedArrays

In [24]:
# ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
# ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

m = ControlMessage(
    message_type=DistributedStreams.start,
    target=4,
    payload=FunctionPayload(
        f=m_f.payload.f,
        in=m_f.payload.in,
        out=m_f.payload.out
    )
)

ControlMessage(DistributedStreams.start, 4, FunctionPayload(test_fn, RemoteChannel{Channel{Int64}}(1, 1, 25), RemoteChannel{Channel{Int64}}(1, 1, 26)))

In [25]:
put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 4
      From worker 2:	Started
      From worker 2:	Worker started for: 4
      From worker 2:	Worker 4 completed


In [26]:
collect!(control_responses)

2-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.started, 4, nothing)
 ControlMessage(DistributedStreams.completed, 4, ReturnPayload(RemoteException(4, CapturedException(UndefVarError(Symbol("#test_fn")), Any[(deserialize_datatype at Serialization.jl:1399, 1), (handle_deserialize at Serialization.jl:867, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814, 1), (#5 at Serialization.jl:973, 1), (ntupleany at ntuple.jl:43, 1), (deserialize_tuple at Serialization.jl:973, 1), (handle_deserialize at Serialization.jl:857, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_msg at messages.jl:87, 1), (#invokelatest#2 at essentials.jl:892 [inlined], 1), (invokelatest at essentials.jl:889 [inlined], 1), (message_handler_loop at process_messages.jl:176, 1), (process_tcp_streams at process_messages.jl:133, 1), (#103 at process_messages.jl:121, 1)]))))

In [27]:
sentinel_running()

true

In [34]:
m = ControlMessage(
    message_type=DistributedStreams.stop,
    target=4,
    func_f=nothing,
    func_in=nothing,
    func_out=nothing
)

put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Stop instruction for 4
      From worker 2:	Stopped


In [25]:
collect!(control_responses)

      From worker 2:	Worker 4 completed


15-element Vector{ControlMessage}:
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, nothing, nothing, nothing)
 ControlMessage(DistributedStreams.completed, 4, noth

      From worker 2:	Worker started for: 4
      From worker 2:	Worker 4 completed
      From worker 2:	Worker started for: 4
      From worker 2:	Worker 4 completed
      From worker 2:	Worker started for: 4
      From worker 2:	Worker 4 completed
      From worker 2:	Worker started for: 4


In [36]:
rmprocs(4)

Task (done) @0x00007214eb11dc30

In [38]:
collect!(control_responses)

ControlMessage[]

In [39]:
workers()

1-element Vector{Int64}:
 2

In [40]:
stop_workers!(control; workers=[2])

      From worker 2:	Sentinel on 2 is shutting down


In [30]:
rmprocs()

Task (done) @0x00007f3c2b37a0e0

In [42]:
istaskdone(t)

false